In [342]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests as re
import os

In [8]:
from google.colab import files
uploaded = files.upload()

Saving restnoteringar-2-0.xml to restnoteringar-2-0 (1).xml


In [359]:
filename = 'restnoteringar-2-0.xml'
xtree = ET.parse(filename)
root = xtree.getroot()

In [360]:
one_channel = ET.tostring(root[0],encoding='utf8').decode('utf8')
print(one_channel)

<?xml version='1.0' encoding='utf8'?>
<ns0:SupplyShortagePublic xmlns:ns0="http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/">
    <ns0:ReferenceNumber>4bdab691-63de-4f1a-9445-3348f7624abd</ns0:ReferenceNumber>
    <ns0:FirstPublicationDate>2018-02-09</ns0:FirstPublicationDate>
    <ns0:LastPublicationDate>2018-02-23</ns0:LastPublicationDate>
    <ns0:PublicPhoneNumber>08-578 135 00</ns0:PublicPhoneNumber>
    <ns0:MedicinalProducts>
      <ns0:MedicinalProduct>
        <ns0:NplId>20050505000047</ns0:NplId>
        <ns0:Packages>
          <ns0:Package>
            <ns0:NplPackId>20050818100014</ns0:NplPackId>
            <ns0:ForecastStartDate>2018-02-09</ns0:ForecastStartDate>
            <ns0:ForecastEndDate>2018-02-20</ns0:ForecastEndDate>
            <ns0:ActualEndDate>2018-02-22</ns0:ActualEndDate>
            <ns0:AdviceSpecification>
              <ns0:AdviceAdditionalInformation ns0:sort="18">
                <ns0:Note>Finns i annan förpackningsstorlek: 10

So it is a nested XML tree

In [289]:
# print essential elements of the tree
print(root.tag)
print(root.attrib)
print(root.text)

{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}SupplyShortagesPublic
{'{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}creationDate': '2021-01-16T06:01:39.6859209+01:00'}

  


In [290]:
# Check key and value for one tree
for child in root[3].iter():
   print(child.tag, child.text)

{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}SupplyShortagePublic 
    
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}ReferenceNumber 3c58b4ad-2ec2-438e-9d17-e5e50edb96f6
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}FirstPublicationDate 2018-02-13
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}LastPublicationDate 2018-05-25
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}PublicPhoneNumber 08-731 28 00
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}PublicEmail infosweden@takeda.com,
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}MedicinalProducts 
      
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}MedicinalProduct 
        
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v1/}NplId 19801003000077
{http://eservices.lakemedelsverket.se/lia/medicinalshortage/publish/v

In [361]:
# Helper funtions
def getName(name):
    # split tag, only take the part after }   
    result = name
    splitArr = name.split('}')
    if len(splitArr) > 1:
        result = splitArr[1]
    return result

def insertKeyValue(root):
    if len(list(root)) > 0:
        for child in root:
            insertKeyValue(child)
    else:
        # Iterate through root to get name (tag) and value (text)
        key = getName(root.tag)
        value = root.text + " "
        # if there are several values, seperate them by \n for pre-processing later
        if key in columItem.keys():
            columItem[key] = columItem[key] + "\n" + value
        else:
            columItem[key] = value


# Initialize
keyvalueList = [] # to get key-value (tag- text) list
maxColumCount = 0 # to create number of columns equal to number of keys later
maxColumIndex = 0 # to get key position later
i = 0

# Find key value in each main child of each Supply Shortage root
for eachSupplyShortagePublic in root:

    columItem = {}
    # insert key value pairs for each single root
    for mainChild in eachSupplyShortagePublic:
        insertKeyValue(mainChild)

    # count number of main child and indexing 
    if len(columItem.keys()) > maxColumCount:
       maxColumCount = len(columItem.keys())
       maxColumIndex = i

    keyvalueList.append(columItem)
    i += 1

# Find key-value for each sub child and append all in a dictionary
dict = {}

if len(keyvalueList) > 0:
    #Get key position by index of keyvalueList 
    mainKeys = keyvalueList[maxColumIndex].keys()
    #Create list of temporary keys
    for tempKey in mainKeys:
        dict[tempKey] = []

    # For loop within keyvalueList
    for i in range(0, len(keyvalueList)):
        column = keyvalueList[i]
        columnKeys = column.keys()

        for key in mainKeys:
            # Put key value into dict if exist
            if key in columnKeys:
                dict[key].append(column[key])
            # else, append empty value
            else:
                dict[key].append("")
else: # if no keyvalue was found
  raise ValueError('There is no data')

# Get dataframe
stockoutse = pd.DataFrame(dict)

In [362]:

stockoutse.head(5)

,ReferenceNumber,FirstPublicationDate,LastPublicationDate,PublicPhoneNumber,PublicEmail,NplId,NplPackId,ForecastStartDate,ForecastEndDate,ActualEndDate,Description,Note,AtcCode
0,4bdab691-63de-4f1a-9445-3348f7624abd,2018-02-09,2018-02-23,08-578 135 00,,20050505000047,20050818100014,2018-02-09,2018-02-20,2018-02-22,,Finns i annan förpackningsstorlek: 10 x 2 ml,
1,43e48426-e824-4689-878e-81da57327341,2018-02-12,2018-04-18,+46-856642572,,19690124000024,19690101100082,2018-02-01,2018-03-05,2018-04-17,,Läkemedel finns i annan beredningsform: Xylopr...,
2,75721bdb-45c9-4c37-9fef-7d6e8e33e867,2018-02-15,2018-05-18,+46-856642572,,19660914000015,19660701100010,2018-02-01,2018-04-30,2018-05-18,,Finns inga utbytbara läkemedel.\nLäkemedel fin...,
3,3c58b4ad-2ec2-438e-9d17-e5e50edb96f6,2018-02-13,2018-05-25,08-731 28 00,"infosweden@takeda.com,",19801003000077,10010101106942 \n10010101106935 \n19960401100021,2018-01-20 \n2018-01-20 \n2018-01-20,2018-05-23 \n2018-05-23 \n2018-05-23,2018-05-24 \n2018-05-24 \n2018-05-24,,"Utbytbara läkemedel: Furosemid Meda, Impugan \...",
4,f560c1e9-8ed1-415d-adc0-503efdbf1b8f,2018-02-14,2018-06-30,+46-856642572,,19930528000047,10010101112950,2018-02-15,2018-04-02,2018-02-26,,Utbytbart läkemedel: Tapin \n[I informationen ...,


We only consider the following variables: 
NplPackId
ForecastStartDate
ActualEndDate	

Because other countries' data don't contain forecastEndDate = Will use ActualEndDate

In [363]:
stockoutse = stockoutse[['NplPackId','ForecastStartDate','ActualEndDate']]


In [373]:
sub_single = stockoutse[~stockoutse['NplPackId'].str.contains("\n")]

multi_rowdf = stockoutse[stockoutse['NplPackId'].str.contains("\n")]


sub1 = multi_rowdf[['NplPackId','ForecastStartDate','ActualEndDate']]

sub1 = sub1.assign(NplPackId=sub1.NplPackId.str.split('\n'))\
          .explode('NplPackId')\
          .reset_index(drop=True)
sub2  = sub1.assign(ForecastStartDate=sub1.ForecastStartDate.str.split('\n'))\
            .explode('ForecastStartDate')\
            .reset_index(drop=True)
sub3  = sub2.assign(ActualEndDate=sub2.ActualEndDate.str.split('\n'))\
            .explode('ActualEndDate')\
            .reset_index(drop=True)

sub4 = sub3[['NplPackId','ForecastStartDate','ActualEndDate']]
sub_multi = sub4.drop_duplicates().reset_index(drop=True)


se = sub_multi.append(sub_single, ignore_index=True)


In [374]:
sub1

,NplPackId,ForecastStartDate,ActualEndDate
0,10010101106942,2018-01-20 \n2018-01-20 \n2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
1,10010101106935,2018-01-20 \n2018-01-20 \n2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
2,19960401100021,2018-01-20 \n2018-01-20 \n2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
3,20060913100079,2018-02-13 \n2018-02-13,2018-04-13 \n2018-04-13
4,20060913100086,2018-02-13 \n2018-02-13,2018-04-13 \n2018-04-13
...,...,...,...
1779,20030917100221,2021-01-11,
1780,20030917100191,2021-01-11,
1781,20130828100089,2021-01-11,
1782,19660701100027,2021-05-15 \n2021-03-01,


In [375]:
sub2

,NplPackId,ForecastStartDate,ActualEndDate
0,10010101106942,2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
1,10010101106942,2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
2,10010101106942,2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
3,10010101106935,2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
4,10010101106935,2018-01-20,2018-05-24 \n2018-05-24 \n2018-05-24
...,...,...,...
10846,20130828100089,2021-01-11,
10847,19660701100027,2021-05-15,
10848,19660701100027,2021-03-01,
10849,19660701100010,2021-05-15,


In [376]:
sub3

,NplPackId,ForecastStartDate,ActualEndDate
0,10010101106942,2018-01-20,2018-05-24
1,10010101106942,2018-01-20,2018-05-24
2,10010101106942,2018-01-20,2018-05-24
3,10010101106942,2018-01-20,2018-05-24
4,10010101106942,2018-01-20,2018-05-24
...,...,...,...
207224,20130828100089,2021-01-11,
207225,19660701100027,2021-05-15,
207226,19660701100027,2021-03-01,
207227,19660701100010,2021-05-15,


In [377]:
se.sort_values(by='ForecastStartDate')

,NplPackId,ForecastStartDate,ActualEndDate
2801,20040922100048,2015-02-20,2019-09-03
2528,20090902100049,2016-09-01,2019-03-21
386,20110209100617,2016-10-03,2019-12-02
387,20110209100549,2016-10-03,2019-12-02
2526,20090902100056,2016-11-24,2019-03-21
...,...,...,...
4736,10010101108175,2021-03-21,
4731,10010101123208,2021-03-31,
4654,19721001100046,2021-05-01,
2328,19660701100027,2021-05-15,


In [340]:
se.to_csv('stockout_se.csv')